# CAI Claude Instant

In [116]:
# %load ./prompt.py
import os
import anthropic
from enum import Enum
from dotenv import load_dotenv
load_dotenv()


anthropic_client = anthropic.Client(os.getenv('ANTHROPIC_API_KEY'))


class Model(Enum):
    claude_v1_0 = "claude-v1.0"
    claude_v1_2 = "claude-v1.2"
    claude_v1_3 = "claude-v1.3"
    claude_v1_3_100k = "claude-v1.3-100k"
    claude_v1_latest = "claude-v1"
    claude_v1_latest_100k = "claude-v1-100k"
    claude_instant_v1_0 = "claude-instant-v1.0"
    claude_instant_v1_1 = "claude-instant-v1.1"
    claude_instant_v1_1_100k = "claude-instant-v1.1-100k"
    claude_instant_v1_latest = "claude-instant-v1"
    claude_instant_v1_latest_100k = "claude-instant-v1-100k"


class Prompt:
    def __init__(
        self,
        human_message: str,
        model: Model = Model.claude_v1_latest,
        temp_0_1: float = 0.5,
        max_tokens_to_sample: int = 1024,
        assistant_prefix: str = None,
        response_prefix: str = None
    ):
        self.human_message = human_message
        self.model = model
        self.temp_0_1 = temp_0_1
        self.max_tokens_to_sample = max_tokens_to_sample
        self.assistant_prefix = assistant_prefix
        self.response_prefix = response_prefix

    @property
    def prompt(self) -> str:
        prompt = f"{anthropic.HUMAN_PROMPT} {self.human_message}{anthropic.AI_PROMPT}"
        if self.assistant_prefix:
            prompt += f" {self.assistant_prefix}"
        return prompt

    def get_response(self) -> str:
        response = anthropic_client.completion(
            prompt=self.prompt,
            stop_sequences=[anthropic.HUMAN_PROMPT],
            model=self.model.value,
            max_tokens_to_sample=self.max_tokens_to_sample,
            temperature=self.temp_0_1,
        )

        text = response['completion']

        if self.response_prefix:
            text = f"{self.response_prefix}{text}"

        return text


In [117]:
model = Model.claude_instant_v1_latest

num_samples = 5

constitution_rules = [
    "Instructions must explicitly describe the tone that the editor adpots.",
    "Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.",
    "Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:\n\n<article>ARTICLE_CONTENT</article>\n\n[Rest of instruction goes here]'.",
    "Instructions which involve multiple steps should be broken down into numbered subtasks.",
    # "Instructions must include explicit examples. For example, if the instruction is to check for spelling errors, it should include an example typo and its correction.",
]
constitution_str = "\n".join(map(lambda x: f"{x[0]+1}. {x[1]}", enumerate(constitution_rules)))


def get_concepts_prompt(n: int) -> Prompt:
    return Prompt(
        model=model,
        temp_0_1=0.7,
        human_message=f"Generate a list of {n} tasks that a Copy Editor might do as part of their job, given a written article. The output of each completed task should be written text. Write each task on a separate, numbered line.",
        assistant_prefix=f"Here are {n} tasks:\n\n1.",
        response_prefix="1.",
    )


def get_naive_response_prompt(concept: str) -> Prompt:
    return Prompt(
        model=model,
        temp_0_1=0.3,
        human_message=f"""I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
{concept}
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. Don't include any preamble, just respond directly with the instruction for the agent.""",
    )


def get_naive_response_prompt_with_constitution(concept: str) -> Prompt:
    return Prompt(
        model=model,
        temp_0_1=0.3,
        human_message=f"""I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
{concept}
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. The instruction must abide by the following rules:

<rules>
{constitution_str}
</rules>

Don't include any preamble, just respond directly with the instruction for the agent.""",
    )


def get_critique_prompt(naive_response: str) -> Prompt:
    return Prompt(
        model=model,
        temp_0_1=0.5,
        human_message=f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
{constitution_str}
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write:

The instruction breaks the following rules:
1. Rule: {constitution_rules[0]} - Reason: ...""",
    )


def get_rewrite_prompt(naive_response: str, critique: str) -> Prompt:
    return Prompt(
        model=model,
        temp_0_1=0.5,
        human_message=f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
{critique}
</issues>

Rewrite the instruction to address these issues. Do not enclose your answer in <instruction> tags.""",
        assistant_prefix="Here is the rewritten instruction:\n\n",
    )


## Generate concepts

In [118]:
concepts_string = get_concepts_prompt(num_samples).get_response()
print(concepts_string)


1. Proofread the article for spelling and grammatical errors.  
2. Check that all names, places, dates, and facts are accurately portrayed. 
3. Ensure consistency in punctuation, capitalization, abbreviations, and numerals.  
4. Verify that quotations are accurate and correctly cited.
5. Make suggestions for reorganizing or restructuring the article for clarity and flow.


In [119]:
import re


def extract_items(text):
    items = []
    lines = text.split('\n')
    for line in lines:
        stripped = line.strip()
        if stripped and re.match(r'^\d+\.\s', stripped):
            items.append(re.sub(r'^\d+\.\s', '', stripped))
    return items


concepts = extract_items(concepts_string)
if len(concepts) != num_samples:
    raise Exception(f"Expected {num_samples} concepts, but got {len(concepts)}")
else:
    print(f"Got {len(concepts)} concepts")


Got 5 concepts


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [121]:
# print(get_naive_response_prompt_with_constitution(concepts[0]).human_message)


In [122]:
# def get_naive_response(concept):
#     return get_claude_response(get_naive_response_prompt_with_constitution(concept))


# # Concurrently (speeds up openai responses, not possible with anthropic)
# # with concurrent.futures.ThreadPoolExecutor() as executor:
# #     naive_responses = list(executor.map(lambda x: get_naive_response(
# #         x[0], x[1]), [(concept, i) for i, concept in enumerate(concepts)]))

# # Sequentially
# naive_responses = []
# for concept in concepts:
#     naive_responses.append(get_naive_response(concept))

# tasksResponsesString = "\n\n\n===\n\n\n".join(
#     [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
# print(tasksResponsesString)


In [123]:
# Anthropic does not allow concurrent requests, so we have to do this sequentially:
naive_responses = []
for concept in concepts:
    naive_responses.append(get_naive_response_prompt(concept).get_response())

tasksResponsesString = "\n\n\n===\n\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
print(tasksResponsesString)


TASK:
Proofread the article for spelling and grammatical errors.

NAIVE RESPONSE:
 Proofread the text for spelling and grammatical errors and suggest corrections.


===


TASK:
Check that all names, places, dates, and facts are accurately portrayed.

NAIVE RESPONSE:
 Check all names, places, dates, and facts for accuracy.


===


TASK:
Ensure consistency in punctuation, capitalization, abbreviations, and numerals.

NAIVE RESPONSE:
 Copy edit all text to ensure consistent:
- Punctuation
- Capitalization  
- Abbreviations
- Numerals


===


TASK:
Verify that quotations are accurate and correctly cited.

NAIVE RESPONSE:
 Verify that all quotations are accurate matches to their original sources and cited correctly.


===


TASK:
Make suggestions for reorganizing or restructuring the article for clarity and flow.

NAIVE RESPONSE:
 Revise the article's organization and structure to improve clarity and the flow of ideas.


Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique

Here's what an example prompt looks like:

In [124]:
print(get_critique_prompt(naive_responses[0]).prompt)




Human: Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the text for spelling and grammatical errors and suggest corrections.
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
1. Instructions must explicitly describe the tone that the editor adpots.
2. Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
3. Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:

<article>ARTICLE_CONTENT</article>

[Rest of instruction goes here]'.
4. Instructions which involve multiple steps should be broken down into numbered subtasks.
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1

Now let's execute the critique prompt on each of the naive responses:

In [125]:
critiques = []
for naive_response in naive_responses:
    critiques.append(get_critique_prompt(naive_response).get_response())

responseCritiqueString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


NAIVE RESPONSE:
 Proofread the text for spelling and grammatical errors and suggest corrections.

CRITIQUE:
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots.
Reason: The instruction does not describe the tone that the editor should adopt when proofreading and suggesting corrections.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
Reason: The instruction does not specify the number of words that the editor's suggestions and corrections should contain.

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:
<article>ARTICLE_CONTENT</article>  
[Rest of instruction goes here].'
Reason: The instruction does not reference any specific article content that the editor should proofread.

4. Rule: Instructions which involve multiple steps

## Rewrite

Here's what an example rewrite prompt looks like:

In [126]:
print(get_rewrite_prompt(naive_responses[0], critiques[0]).prompt)




Human: Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the text for spelling and grammatical errors and suggest corrections.
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots.
Reason: The instruction does not describe the tone that the editor should adopt when proofreading and suggesting corrections.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
Reason: The instruction does not specify the number of words that the editor's suggestions and corrections should contain.

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:
<article>ARTICLE_CONTENT</article>  
[Rest of instruct

Now let's execute the rewrite prompt on each of the naive responses:

In [127]:
rewrites = []
for i, (naive_response, critique) in enumerate(zip(naive_responses, critiques)):
    rewrites.append(get_rewrite_prompt(naive_response, critique).get_response())

rewriteString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nREWRITE:\n{rewrite}" for response, rewrite in zip(naive_responses, rewrites)])
print(rewriteString)


NAIVE RESPONSE:
 Proofread the text for spelling and grammatical errors and suggest corrections.

REWRITE:
 Proofread the following article for spelling and grammatical errors while adopting a polite and helpful tone:  

Here is an article:  
 <article>ARTICLE_CONTENT</article>

1. Check for spelling mistakes and typos. Suggest corrections in approximately 2-3 words per correction.  

2. Check for grammatical errors such as incorrect verb forms, pronoun issues, and punctuation mistakes. Suggest corrections in approximately  2-3 words per correction.

3. Provide a maximum of 5 overall corrections and suggestions to improve the clarity and readability of the text while maintaining the original meaning.


===


NAIVE RESPONSE:
 Check all names, places, dates, and facts for accuracy.

REWRITE:
 Check the attached article for accuracy. Adopt a polite yet firm tone in your edits. Provide a 200 word summary of your fact checks and corrections.

1. Check all names for accuracy.   

2. Verify t

## Rewrites as few-shot

This is to mimic a retrained model

In [128]:
few_shot_prompt = ''
for i, (concept, rewrite) in enumerate(zip(concepts, rewrites)):
    few_shot_prompt += f"{anthropic.HUMAN_PROMPT} {get_naive_response_prompt(concept).human_message}"
    few_shot_prompt += f"{anthropic.AI_PROMPT}"
    if i < len(concepts) - 1:
        few_shot_prompt += f" {rewrite}"

print(few_shot_prompt)




Human: I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
Proofread the article for spelling and grammatical errors.
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. Don't include any preamble, just respond directly with the instruction for the agent.

Assistant:  Proofread the following article for spelling and grammatical errors while adopting a polite and helpful tone:  

Here is an article:  
 <article>ARTICLE_CONTENT</article>

1. Check for spelling mistakes and typos. Suggest corrections in approximately 2-3 words per correction.  

2. Check for grammatical errors such as incorrect verb forms, pronoun issues, and punctuation mistakes. Suggest corrections in approximately  2-3 words per correction.

3. Provide a maximum of 5 overall corrections and suggestions to improve the clarity and readability of the text while maintaining the original meaning.

H

In [129]:
few_shot_response = anthropic_client.completion(
    prompt=few_shot_prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=model.value,
    max_tokens_to_sample=1024,
    temperature=0.5,
)

text = few_shot_response['completion']
print(text)


 Here is an article:  

<article>ARTICLE_CONTENT</article>

Suggest edits in around 200 words to improve the flow, structure and clarity of the attached article. Provide a brief rationale for each suggestion. Adopt a helpful yet professional tone.

1. Propose reorganizing or restructuring sections to improve the logical progression of ideas.

2. Suggest combining or splitting paragraphs where appropriate.

3. Recommend moving, deleting or adding transitional phrases for better cohesion.  

4. Flag any sentences that are unclear, wordy or redundant and propose rewriting for clarity and conciseness.
